# Testando Yolov8

Dentro de este notebook procederemos a testar un conjunto de imagens com o modelo Yolov8.

Para ello, realizaremos lo siguiente:
- Cortaremos un ortomosáico -igual que hicimos para el conjunto de training.
- Prepararemos las etiquetas de dicho ortomosaico.
- Realizaremos la predicción con el modelo Yolov8.

In [1]:
import utils.yolo_fun as yolo_fun
import utils.img_fun as img_fun
import os
import pandas as pd
from tqdm import tqdm  
import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioIOError
import numpy as np
import shutil
from sklearn.model_selection import train_test_split



coords_dir_sin_normalizar = os.path.join('test', 'labels_sin_normalizar')
coords_dir_normalized = os.path.join('test', 'labels_normalized')
subrecortes_dir = os.path.join('test', 'img')


os.makedirs(coords_dir_sin_normalizar, exist_ok=True)
os.makedirs(coords_dir_normalized, exist_ok=True)
os.makedirs(subrecortes_dir, exist_ok=True)


#path_doctorado = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos\\recortes'

path_doctorado = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\pinguiton\\ortho_dic_5000'



In [2]:

#! Haz el recorte con las coordenadas de yolo_coords, pero crea los txt independientes con test_coords
orthomosiac_coords = os.path.join('test', 'test_coords.csv')


# PARTE 0: RECORTE DE IMÁGENES PARA EL TEST
contador = 0
for img in os.listdir(path_doctorado):
    # contador += 1
    # if contador == 5: break
    
    try:
        # PARTE 1: Cargar la imagen y recortarla en imágenes más pequeñas de aproximadamente 500x500 píxeles
        print(f"\n\nRecortando imagen {img}...")
        print('_________________________________________________________')

        img_name = img.split('.')[0]
        

        tiff_file = os.path.join(path_doctorado, img_name + '.tif')


        # Sacamos un diccionario con toda la información de la imagen
        img_info = img_fun.get_img_info(tiff_file)
        WIDTH = img_info["width"]
        HEIGHT = img_info["height"]
        TOP_LEFT = img_info["top_left"]
        BOTTOM_RIGHT = img_info["bottom_right"]
        min_x, max_y = img_info['top_left']
        max_x, min_y = img_info['bottom_right']

        img_fun.crop_tile_into_subrecortes(
            tiff_file = tiff_file, 
            output_dir = subrecortes_dir, 
            coords_csv = orthomosiac_coords,
            tile_size = 640,
            overlap=0
        )
        
     
    except RasterioIOError as e:
        print(f"Error al cargar la imagen con rasterio: {e}")
        continue
    except FileNotFoundError as e:
        print(f"Error: {e}")
        continue

    



Recortando imagen imagen-15-8.tif...
_________________________________________________________
Metadata:
---------
driver: GTiff
dtype: uint8
nodata: None
width: 5000
height: 5000
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.21|
| 0.00,-0.00,-62.31|
| 0.00, 0.00, 1.00|
blockxsize: 256
blockysize: 256
tiled: True
compress: lzw
interleave: pixel

Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.213980474998074, -62.307177528123795)
BOTTOM RIGHT: (-59.212102454998075, -62.3080516081238)
Recorte: (0, 0), Coordenadas encontradas: 0
Recorte vacío: imagen-15-8_0_0.tiff
Recorte: (0, 640), Coordenadas encontradas: 0
Recorte vacío: imagen-15-8_0_640.tiff
Recorte: (0, 1280), Coordenadas encontradas: 0
Recorte vacío: imagen-15-8_0_1280.tiff
Recorte: (0, 1920), Coordenadas encontradas: 0
Recorte vacío: imagen-15-8_0_1920.tiff
Recorte: (0, 2560), Coordenadas encontradas: 0
Recorte vacío: imagen-15-8_0_2560.tiff
Recorte: (0, 3200), Coordenadas encontradas: 0
Recorte vacío: imagen-15-

In [6]:
# PARTE 3: ASIGNACIÓN DE LABELS EN TXT A CADA SUBRECORTE

orthomosiac_coords = os.path.join('test', 'test_coords.csv')
yolo_fun.generar_txt_yolo(
    subrecorte_dir = subrecortes_dir, 
    csv_file = orthomosiac_coords, 
    coords_dir = coords_dir_sin_normalizar
)

# Eliminamos todas las etiquetas o imágenes que no estén en la carpeta homóloga



Generando archivos .txt: 100%|██████████| 1432/1432 [00:26<00:00, 54.04it/s]

Archivos .txt generados en test\labels_sin_normalizar


# Elminamos imágenes que no tengan etiquetas

In [7]:

# Rutas de las carpetas
img_folder = subrecortes_dir
label_folder = coords_dir_sin_normalizar

# Listar los archivos en las carpetas
img_files = [f for f in os.listdir(img_folder) if f.endswith('.tiff')]
label_files = [f.replace('.txt', '') for f in os.listdir(label_folder) if f.endswith('.txt')]

# Verificar y borrar las imágenes que no tienen el archivo .txt correspondiente
for img_file in img_files:
    img_name = img_file.replace('.tiff', '')
    if img_name not in label_files:
        # Si no hay archivo .txt correspondiente, eliminar la imagen .tiff
        img_path = os.path.join(img_folder, img_file)
        os.remove(img_path)
        print(f"Imagen eliminada: {img_file}")

Imagen eliminada: imagen-1-2_0_0.tiff
Imagen eliminada: imagen-1-2_0_1280.tiff
Imagen eliminada: imagen-1-2_0_1920.tiff
Imagen eliminada: imagen-1-2_0_2560.tiff
Imagen eliminada: imagen-1-2_0_3200.tiff
Imagen eliminada: imagen-1-2_0_3840.tiff
Imagen eliminada: imagen-1-2_0_4480.tiff
Imagen eliminada: imagen-1-2_0_640.tiff
Imagen eliminada: imagen-1-2_1280_0.tiff
Imagen eliminada: imagen-1-2_1280_1280.tiff
Imagen eliminada: imagen-1-2_1280_1920.tiff
Imagen eliminada: imagen-1-2_1280_2560.tiff
Imagen eliminada: imagen-1-2_1280_3200.tiff
Imagen eliminada: imagen-1-2_1280_640.tiff
Imagen eliminada: imagen-1-2_1920_0.tiff
Imagen eliminada: imagen-1-2_1920_1280.tiff
Imagen eliminada: imagen-1-2_1920_1920.tiff
Imagen eliminada: imagen-1-2_1920_2560.tiff
Imagen eliminada: imagen-1-2_1920_4480.tiff
Imagen eliminada: imagen-1-2_1920_640.tiff
Imagen eliminada: imagen-1-2_2560_0.tiff
Imagen eliminada: imagen-1-2_2560_1280.tiff
Imagen eliminada: imagen-1-2_2560_1920.tiff
Imagen eliminada: imagen-1-

In [8]:


# Iteramos para normalizar las coordenadas de cara archivo.txt
for file in os.listdir(coords_dir_sin_normalizar):
    df_sin_normalizar = pd.read_csv(os.path.join(coords_dir_sin_normalizar, file), sep=' ', header=None)
    name_subrecorte = os.path.splitext(file)[0]
    subrecorte_file = os.path.join(subrecortes_dir, f"{name_subrecorte}.tiff")

    print(f"Normalizando archivo {file}...")
    coords_file = os.path.join(coords_dir_sin_normalizar, file)
    output_file = os.path.join(coords_dir_normalized, file)
    
    try:
        yolo_fun.normalize_yolo_coords(
            tiff_file = subrecorte_file,
            coords_sin_normalizar = df_sin_normalizar, 
            output_file = output_file, 
        )
    except RasterioIOError as e:
        print(f"Error al cargar la imagen con rasterio: {e}")
        # Eliminamos tanto el archivo de coordenadas como la imagen que hayan dado error
        if os.path.exists(subrecorte_file):
            os.remove(subrecorte_file)
            print(f"Imagen {subrecorte_file} eliminada.")
        if os.path.exists(coords_file):
            os.remove(coords_file)
            print(f"Archivo de coordenadas {coords_file} eliminado.")


Normalizando archivo imagen-1-2_1280_3840.txt...
Normalizando archivo imagen-1-2_1280_4480.txt...
Normalizando archivo imagen-1-2_1920_3200.txt...
Normalizando archivo imagen-1-2_1920_3840.txt...
Normalizando archivo imagen-1-2_2560_4480.txt...
Normalizando archivo imagen-1-2_640_4480.txt...
Normalizando archivo imagen-1-4_3840_3200.txt...
Normalizando archivo imagen-1-4_3840_3840.txt...
Normalizando archivo imagen-1-4_3840_4480.txt...
Normalizando archivo imagen-1-4_4480_2560.txt...


c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0.046875
1    0.046875
2    0.046875
3    0.046875
4    0.046875
5    0.046875
6    0.046875
Name: 3, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 3] = coords_sin_normalizar.iloc[:, 3] / width  # Normalización del ancho
c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0.046875
1    0.046875
2    0.046875
3    0.046875
4    0.046875
5    0.046875
6    0.046875
Name: 4, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 4] = coords_sin

Normalizando archivo imagen-1-4_4480_3200.txt...
Normalizando archivo imagen-1-4_4480_3840.txt...
Normalizando archivo imagen-1-4_4480_4480.txt...
Normalizando archivo imagen-11-9_0_0.txt...
Normalizando archivo imagen-11-9_0_1920.txt...
Normalizando archivo imagen-11-9_0_2560.txt...
Normalizando archivo imagen-11-9_1280_0.txt...
Normalizando archivo imagen-11-9_1280_1280.txt...
Normalizando archivo imagen-11-9_1280_640.txt...


c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.046875
1     0.046875
2     0.046875
3     0.046875
4     0.046875
        ...   
70    0.046875
71    0.046875
72    0.046875
73    0.046875
74    0.046875
Name: 3, Length: 75, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 3] = coords_sin_normalizar.iloc[:, 3] / width  # Normalización del ancho
c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.057692
1     0.057692
2     0.057692
3     0.057692
4     0.057692
        ...   
70    0.057692
71    0.057692
72    0.057692
73    0.057692
74    0.057692
Name: 4, Leng

Normalizando archivo imagen-11-9_1920_0.txt...
Normalizando archivo imagen-11-9_1920_640.txt...
Normalizando archivo imagen-11-9_2560_0.txt...
Normalizando archivo imagen-11-9_2560_1280.txt...
Normalizando archivo imagen-11-9_2560_640.txt...
Normalizando archivo imagen-11-9_3200_1280.txt...
Normalizando archivo imagen-11-9_3200_640.txt...
Normalizando archivo imagen-11-9_640_0.txt...
Normalizando archivo imagen-11-9_640_640.txt...
Normalizando archivo imagen-16-10_1280_0.txt...


...

c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.046875
1     0.046875
2     0.046875
3     0.046875
4     0.046875
5     0.046875
6     0.046875
7     0.046875
8     0.046875
9     0.046875
10    0.046875
11    0.046875
12    0.046875
13    0.046875
14    0.046875
15    0.046875
16    0.046875
17    0.046875
18    0.046875
19    0.046875
20    0.046875
21    0.046875
22    0.046875
23    0.046875
24    0.046875
25    0.046875
26    0.046875
27    0.046875
28    0.046875
29    0.046875
30    0.046875
31    0.046875
Name: 3, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 3] = coords_sin_normalizar.iloc[:, 3] / width  # Normalización del ancho
c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.p

Normalizando archivo imagen-5-2_3840_1280.txt...
Normalizando archivo imagen-5-2_3840_1920.txt...
Normalizando archivo imagen-5-2_3840_2560.txt...
Normalizando archivo imagen-5-2_3840_3200.txt...
Normalizando archivo imagen-5-2_3840_3840.txt...
Normalizando archivo imagen-5-2_3840_640.txt...
Normalizando archivo imagen-5-2_4480_1280.txt...
Normalizando archivo imagen-5-2_4480_1920.txt...
Normalizando archivo imagen-5-2_4480_2560.txt...


c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.046875
1     0.046875
2     0.046875
3     0.046875
4     0.046875
5     0.046875
6     0.046875
7     0.046875
8     0.046875
9     0.046875
10    0.046875
11    0.046875
12    0.046875
Name: 3, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 3] = coords_sin_normalizar.iloc[:, 3] / width  # Normalización del ancho
c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.057692
1     0.057692
2     0.057692
3     0.057692
4     0.057692
5     0.057692
6     0.057692
7     0.057692
8     0.057692
9     0.057692
10    0.05

Normalizando archivo imagen-5-2_4480_3200.txt...
Normalizando archivo imagen-5-2_4480_3840.txt...
Normalizando archivo imagen-5-2_4480_640.txt...
Normalizando archivo imagen-5-2_640_0.txt...
Normalizando archivo imagen-5-2_640_1280.txt...
Normalizando archivo imagen-5-2_640_3840.txt...
Normalizando archivo imagen-5-2_640_4480.txt...
Normalizando archivo imagen-6-8_0_1280.txt...
Normalizando archivo imagen-6-8_0_640.txt...
Normalizando archivo imagen-6-8_640_1280.txt...
Normalizando archivo imagen-6-8_640_1920.txt...
Normalizando archivo imagen-7-6_640_1920.txt...


c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     0.046875
1     0.046875
2     0.046875
3     0.046875
4     0.046875
5     0.046875
6     0.046875
7     0.046875
8     0.046875
9     0.046875
10    0.046875
11    0.046875
12    0.046875
13    0.046875
14    0.046875
15    0.046875
16    0.046875
17    0.046875
18    0.046875
19    0.046875
20    0.046875
21    0.046875
22    0.046875
23    0.046875
24    0.046875
Name: 3, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coords_sin_normalizar.iloc[:, 3] = coords_sin_normalizar.iloc[:, 3] / width  # Normalización del ancho
c:\Users\Administrador.CRISASUSESTUDIO\Desktop\projects\CountingPenguins\utils\yolo_fun.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future erro

In [18]:


# PARTE 4: CLASIFICAR CONJUNTOS DE TRAIN Y VAL

# Listar todas las imágenes y sus respectivos archivos de coordenadas
images = [img for img in os.listdir(subrecortes_dir) if img.endswith('.tiff')]
annotations = [os.path.join(coords_dir_normalized, f'{os.path.splitext(img)[0]}.txt') for img in images]

# Filtrar solo las imágenes que tienen un archivo de coordenadas no vacío
valid_images = []
valid_annotations = []
for img, txt_path in zip(images, annotations):
    if os.path.exists(txt_path) and os.path.getsize(txt_path) > 0:
        valid_images.append(img)
        valid_annotations.append(txt_path)

# Dividir las imágenes y etiquetas en conjuntos de entrenamiento (80%) y validación (20%)
train_images, val_images, train_annotations, val_annotations = train_test_split(
    valid_images, valid_annotations, test_size=0.2, random_state=42
)

# Crear directorios de salida si no existen
os.makedirs('./datasets/penguin_dataset/images/train', exist_ok=True)
os.makedirs('./datasets/penguin_dataset/images/val', exist_ok=True)
os.makedirs('./datasets/penguin_dataset/labels/train', exist_ok=True)
os.makedirs('./datasets/penguin_dataset/labels/val', exist_ok=True)

# Copiar archivos al conjunto de entrenamiento
for img, txt in zip(train_images, train_annotations):
    shutil.copy(os.path.join(subrecortes_dir, img), './datasets/penguin_dataset/images/train')
    shutil.copy(txt, './datasets/penguin_dataset/labels/train')

# Copiar archivos al conjunto de validación
for img, txt in zip(val_images, val_annotations):
    shutil.copy(os.path.join(subrecortes_dir, img), './datasets/penguin_dataset/images/val')
    shutil.copy(txt, './datasets/penguin_dataset/labels/val')

print(f"Se procesaron {len(train_images)} imágenes para entrenamiento y {len(val_images)} imágenes para validación.")

Se procesaron 1120 imágenes para entrenamiento y 281 imágenes para validación.
